In [1]:
import torch as tc
from transformers import AutoModelForCausalLM, AutoTokenizer\

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
def to_device(inputs, device):
    return {k: (v.to(device) if tc.is_tensor(v) else v) for k, v in inputs.items()}

In [3]:
device = tc.device("mps") if tc.backends.mps.is_available() else tc.device("cpu")

model_name = "gpt2"
prompt = "The future of physics is"

model = AutoModelForCausalLM.from_pretrained(
    model_name
).to(device)

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
inputs = tokenizer(
    prompt,
    return_tensors="pt"
)
inputs = to_device(inputs, device)
print(inputs)

{'input_ids': tensor([[  464,  2003,   286, 11887,   318]], device='mps:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1]], device='mps:0')}


In [5]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"
model.config.pad_token_id = tokenizer.pad_token_id

with tc.no_grad():
    outputs = model.generate(
        **inputs, 
        max_new_tokens=50,
        pad_token_id=tokenizer.pad_token_id
    )

for seq in outputs: 
    print(tokenizer.decode(seq, skip_special_tokens=True))

The future of physics is in the hands of the people who are going to make it happen.

"We're going to have to make sure that we're not just going to be able to do this, but we're going to have to make sure that we're
